Imports

In [ ]:
# lib reloading
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# python standard packages
import logging
import re
import time

# third-party packages
import keyring
from selenium.webdriver.support.select import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

# in project packages
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))  # add parent folder to paths
from utilities.asset_store_wrapper import AssetStoreWrapper, AssetStoreAssetButtonError
from project_settings import ProjectSettings

# keyring needed to be setup earlier
# keyring.set_password("credential_name", "user_name", "password")
# or
# add manualy in windows credential manager in generic credential section

# logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

Asset Store Login and go to assets page

In [ ]:
asset_store = AssetStoreWrapper()
asset_store.login_to_asset_store((ProjectSettings.UNITY_CREDENTIALS_NAME, ProjectSettings.UNITY_CREDENTIALS_EMAIL))
asset_store.open_free_not_added_assets()
assets = asset_store.get_assets_from_page_as_collection()

Do something with assets in case of tests

In [ ]:
for asset in assets:
    logging.info(';'.join(asset.text.split('\n')))  # convert to csv line

Close

In [ ]:
asset_store.chrome.quit()